<!--img src="images\cloudflow.png" width=1400 /-->

# Introduction


## How to use this notebook
This example runs Sima in the cloud based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. This notebook assumes that all tooling is in place see the installation section of the notebook for [local execution](workflowCoreDemoSE28SestraAndWasim.ipynb#installation)

This note book should be run in the following way:
1. [Initialize Workflow](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up custom user parameters](#custom) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Set up OneWorkflow client](#builder).
2. [Prepare Sima Input](#prepare) sets up reading of the environment input file and build Sima input accordingly. If the input file changes OO needs to change this section.
3. [Run analysis](#run) shall be run each time a new analysis needs to be run.
4. [Run Wasim and Sestra](#runwasim) run Wasim and Sestra using results from above Sima run.


# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Set up custom user parameters <a id='custom'></a>

In [10]:
# local workspace, all results will be put here after local or cloud runs
import os
workspacePath = r'C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud'
# location of common files for all analysis, has to be below workspacePath
workspaceId = "SE28"
loadcase_file = f"{workspacePath}\\test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"
cloud_run = True


C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud


## Set up OneWorkflow client <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [2]:
from oneWorkflowToolBox import *
workflow_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, tmp=r"c:\OneWorkflowTemp")
workspace = workflow_client.one_workflow_config.workspace_config
commonfiles_folder = workflow_client.common_directory
results_folder = workflow_client.results_directory
#If running locally the code below will also start the local workflow host.
if (cloud_run):
    workflow_client.login()


c:\OneWorkflowTemp


## Upload common files for the job <a id='upload'></a>
This step uploads all common files in folder *commonFilesDirectory*  to the job. Only needed to run if new common files are to be uploaded or workspace changed.

In [5]:
from dnv.onecompute.directory_client import FileOptions
try:
    workflow_client.upload_common_files(FileOptions(
        # max_size_bytes=124_000,
        patterns=["*.txt","*.py"],
        overwrite=True))
except Exception as e:
    print(e)
    print("Ignore this error message if the files are already present.")


Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\requirements.txt to SE28/CommonFiles/requirements.txt
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\copyFiles.py to SE28/CommonFiles/copyFiles.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\postprocessing.py to SE28/CommonFiles/postprocessing.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\CommonFiles\test.py to SE28/CommonFiles/test.py


# Run analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [4]:
from oneWorkflowToolBox import run_workflow_async
from SimaHelper import *
import json
"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()
#Sima path must be specified
sima_settings= SimaSettings(sima_exe_path=r'C:\Program Files\DNV\Sima V4.4-00')
sima_work_unit = SimaTaskCreator(sima_settings, workflow_client).get_sima_work_unit(loadcase_file, stask_file)
# Create Parallel Work Unit and Job
job = workflow_client.create_job(sima_work_unit)

job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
# Run workflow
downloadOptions = FileOptions(
    max_size_bytes=11124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg"])
await run_workflow_async(job, workflow_client, downloadOptions)


Error: load cases directory C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud\LoadCases does not exist.
The work item 9f2f3d37-8bb0-4b7a-995f-71b795d2097c message is 'Pending'
The work item test001 message is 'Pending'
The work item test002 message is 'Pending'
The work item test003 message is 'Pending'
The progress of the job is '0%'. The message is ''
The work item test002 message is ''
The work item test001 message is '2023-06-05 08:24:25,876 [main] INFO  SIMALog - Logging to: /mnt/batch/tasks/workitems/8624594d-3a95-429c-9ba4-14e8a7ce23bc/job-1/test001/wd/LoadCases/test001/sima.log - at no.marintek.sima.util.SIMALog.initializeLogging(SIMALog.java:33)'
The work item test002 message is 'Starting the application:  FileName: /mnt/batch/tasks/applications/simalinux4.4.02022-10-24-19-48/sre arguments: --consoleLog  --log-level ALL --data . --import file=/mnt/batch/tasks/workitems/8624594d-3a95-429c-9ba4-14e8a7ce23bc/job-1/jobpreparation/wd/Common

# Run Wasim and Sestra <a id='runwasim'></a>

In [6]:
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
from dnv.sesam.wasim_command import WasimStruCommand,  WasimSetupCommand
from dnv.sesam.sestra_command import SestraCommand
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import run_workflow_async
import json
import os
workspace.results_directory = "Results"
load_cases = ["test001", "test002"]
workflow_windows_client = one_workflow_client(
    workspaceId, workspacePath, cloud_run, "", Platform.Windows)
os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {'FMT': topSuperElement, 'topsel': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        results_directory: str, load_cases: str) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue

        casedict = case.to_dict()
       
        load_case_result_files_dir = os.path.join(results_directory, casename)
        cmd = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        work_unit = (
            WorkUnit(cmd,  f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"
                                                                         ,"**/*.LIS", "**/*.MLG"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)


workflow_windows_client.login()
work_unit = run_wasim_and_sestra_using_results_from_sima(
    workspace.results_directory, load_cases)

downloadOptions = FileOptions(
    #max_size_bytes=1001124_000,
    patterns=["**/*.txt", "**/*.lis", "**/*.mlg", "**/*.sin"])
await workflow_windows_client.run_workflow_async(
    work_unit,
    work_item_status_changed=work_item_status_changed_callback(
        workflow_windows_client, downloadOptions),
    log_job=True
)



skipping test003
{
    "$type": "DNVGL.One.Compute.Core.FlowModel.Job, DNVGL.One.Compute.Core",
    "properties": {},
    "JobId": "8f7dfa5e-9c3a-49ad-b673-b8ed80d7c702",
    "ServiceName": "ImproveFlowWorker",
    "PoolId": "",
    "JobPreparationWork": {
        "$type": "DNVGL.One.Compute.Core.FlowModel.WorkUnit, DNVGL.One.Compute.Core",
        "properties": {},
        "id": "40dbd4d5-2db0-4b90-a025-81ba1b8bc795",
        "TaskRole": 2,
        "Data": {
            "Version": 1
        },
        "Command": "OneWorkflowWorkerHost",
        "InputFileSpecifications": [
            {
                "$type": "DNVGL.One.Compute.Core.FlowModel.DataTransfer.FileTransferSpecification, DNVGL.One.Compute.Core",
                "SourceSpecification": {
                    "$type": "DNVGL.One.Compute.Core.FlowModel.DataTransfer.BlobDirectorySpecification, DNVGL.One.Compute.Core",
                    "Directory": "SE28\\CommonFiles",
                    "ContainerUrl": "https://tstpeuwst02

'8f7dfa5e-9c3a-49ad-b673-b8ed80d7c702'

# Post processing  <a id='postprocessing'></a>

In [12]:
from IPython.display import display
import numpy as np
import pandas as pd
import glob
from ipywidgets import interactive

lc = 11
dataFrames = {}
print(workspacePath)
for folder in glob.glob(f"{workspacePath}\\Results\\*"):
    test_name = folder.split("\\")[-1]
    try:
        data = np.loadtxt(f"{folder}\\postprocessedresultsLC{lc}.txt")
        dispdata = {
            "x-coord": data[:, 0],
            "total-disp":  np.sqrt(data[:, 3]**2+data[:, 4]**2+data[:, 5]**2)
        }
        dataFrames[test_name] = pd.DataFrame(dispdata)
    except:
        print("Failed loading data for test :" + test_name)


def multiplot(resultcase):
    dataFrames[resultcase].plot(
        x="x-coord", y="total-disp", kind="scatter", figsize=(15, 15))


interactive_plot = interactive(
    multiplot, resultcase=dataFrames.keys())
interactive_plot


C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\WorkspaceCloud
Failed loading data for test :post_rerun_test001
Failed loading data for test :post_rerun_test002
Failed loading data for test :test003


interactive(children=(Dropdown(description='resultcase', options=('test001', 'test002'), value='test001'), Out…